## Packages

In [3]:
import os
import pandas as pd
import numpy as np
import pyBigWig
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from scipy.stats import spearmanr
import xgboost as xgb

## Data Loading & Preprocessing

We run big_wig_to_bw_renaming.sh to rename all data files to .bw format

In [4]:
# The following script is available to execute as a .sh file to rename all the data files format to .bw
#
# #!/bin/bash
#
# # Check if the input variable $Path_to_ML4G_Project_1_Data is set
# if [ -z "$Path_to_ML4G_Project_1_Data" ]; then
#     echo 'Missing input variable $Path_to_ML4G_Project_1_Data. Please run big_wig_to_bw_renaming.sh with the correct path as an argument.'
#     exit 1
# fi
#
# # Navigate to the base data directory
# cd "$Path_to_ML4G_Project_1_Data" || exit
#
# # Find and rename all bigWig files to .bw
# find . -type f \( -iname "*.bigwig" -o -iname "*.bigWig" -o -iname "*.BigWig" \) -print0 | while IFS= read -r -d '' file; do
#     # Get the directory and base name of the file
#     dir=$(dirname "$file")
#     base=$(basename "$file")
#     # Construct the new file name with .bw extension
#     newbase="${base%.*}.bw"
#     newfile="$dir/$newbase"
#     # Rename the file
#     mv "$file" "$newfile"
#     echo "Renamed '$file' to '$newfile'"
# done

In [5]:
# Load bigwig files and extract features for each gene 
def load_features(file_path, gene_df, window_length, num_bins):
    
    print(f"Loading this bigWig file: {file_path}")
    bw = pyBigWig.open(file_path)
    features = []
    half = window_length // 2

    for idx, row in gene_df.iterrows():
        chromosome = row['chr'] 
        strand = row['strand']
        tss = int(row['TSS_start'] if strand == '+' else row['TSS_end'])
        begin = max(0, tss - half)  #to ensure not extending window to non-negative values
        end = tss + half
        bins = np.linspace(begin, end, num_bins + 1, dtype=int)
        vals = []
        for i in range(num_bins):
            bin_begin = bins[i]
            bin_end = bins[i + 1]
          
            bin_vals = bw.stats(chromosome, bin_begin, bin_end, type='mean')[0]
            if bin_vals is None:
                bin_vals = 0
            vals.append(bin_vals)
        features.append(vals)

    bw.close()
    features_df = pd.DataFrame(features)
    features_df.columns = [f'bin_{i}' for i in range(num_bins)]
    features_df['gene_name'] = gene_df['gene_name'].values
    return features_df

# Merge CAGE info and expression data 
def load_cage(info_path, expression_path=None):
    
    info_df = pd.read_csv(info_path, sep='\t')
    if expression_path:
        expression_df = pd.read_csv(expression_path, sep='\t')
        merged_df = pd.merge(info_df, expression_df, on='gene_name')
    else:
        merged_df = info_df
    return merged_df

# Process a cell line with features 
def process_cell_line(line, hist_marks, window_length, num_bins, data, data_type='train'):
    if data_type == 'train':
        info_file = os.path.join(data, 'CAGE-train', 'CAGE-train', f'{line}_train_info.tsv')
        expression_file = os.path.join(data, 'CAGE-train', 'CAGE-train', f'{line}_train_y.tsv')
    elif data_type == 'validation':
        info_file = os.path.join(data, 'CAGE-train', 'CAGE-train', f'{line}_val_info.tsv')
        expression_file = os.path.join(data, 'CAGE-train', 'CAGE-train', f'{line}_val_y.tsv')
    elif data_type == 'test':
        info_file = os.path.join(data, 'CAGE-train', 'CAGE-train', f'{line}_test_info.tsv')
        expression_file = None

    gene_df = load_cage(info_file, expression_file)
    final_df = gene_df.copy()

    # Extract features for each histone mark 
    for mark in hist_marks:
        if mark == 'DNase':
            mark_df = 'DNase-bigwig'
        else:
            mark_df = f'{mark}-bigwig'

        file_extension = '.bw'
        bw_file = os.path.join(data, mark_df, f'{line}{file_extension}')

        # Extract feature and include in df 
        features_df = load_features(bw_file, gene_df, window_length, num_bins)
        feature_cols = [f'{mark}_bin_{i}' for i in range(num_bins)]
        features_df.columns = feature_cols + ['gene_name']
        final_df = pd.merge(final_df, features_df, on='gene_name')

    return final_df


# Define parameters: here 2000 and 20 determined to perform best 
window_length = 2000
num_bins = 20           
hist_marks = ['H3K4me1', 'H3K4me3', 'H3K27ac', 'DNase']
data = '/Users/gonuni/Desktop/College/CBB/3rd_Semester/ML4Genomics/Projects/Project_1/ML4G_Project_1_Data/'

# Process training data
X1_train_data = process_cell_line('X1', hist_marks, window_length, num_bins, data, data_type='train')
X2_train_data = process_cell_line('X2', hist_marks, window_length, num_bins, data, data_type='train')

# Modify gene names so there is no overlap 
X1_train_data['gene_name'] = X1_train_data['gene_name'] + '_X1'
X2_train_data['gene_name'] = X2_train_data['gene_name'] + '_X2'

# training data
X_train_full = pd.concat([X1_train_data, X2_train_data], ignore_index=True)

if not os.path.exists('./Data'):
    os.makedirs('./Data')
    
X_train_full.to_csv('./Data/X_train.tsv', sep='\t', index=False)

# validation data
X1_val_data = process_cell_line('X1', hist_marks, window_length, num_bins,data, data_type='validation')
X2_val_data = process_cell_line('X2', hist_marks, window_length, num_bins,data, data_type='validation')

# Modify gene names again so there is no overlap 
X1_val_data['gene_name'] = X1_val_data['gene_name'] + '_X1'
X2_val_data['gene_name'] = X2_val_data['gene_name'] + '_X2'

# validation data
X_validation = pd.concat([X1_val_data, X2_val_data], ignore_index=True)
X_validation.to_csv('./Data/X_validation.tsv', sep='\t', index=False)

# test data (X3)
X3_test_data = process_cell_line('X3', hist_marks, window_length, num_bins, data, data_type='test')
X3_test_data.to_csv('./Data/X_3_test.tsv', sep='\t', index=False)

Loading this bigWig file: /Users/gonuni/Desktop/College/CBB/3rd_Semester/ML4Genomics/Projects/Project_1/ML4G_Project_1_Data/H3K4me1-bigwig/X1.bw
Loading this bigWig file: /Users/gonuni/Desktop/College/CBB/3rd_Semester/ML4Genomics/Projects/Project_1/ML4G_Project_1_Data/H3K4me3-bigwig/X1.bw
Loading this bigWig file: /Users/gonuni/Desktop/College/CBB/3rd_Semester/ML4Genomics/Projects/Project_1/ML4G_Project_1_Data/H3K27ac-bigwig/X1.bw
Loading this bigWig file: /Users/gonuni/Desktop/College/CBB/3rd_Semester/ML4Genomics/Projects/Project_1/ML4G_Project_1_Data/DNase-bigwig/X1.bw
Loading this bigWig file: /Users/gonuni/Desktop/College/CBB/3rd_Semester/ML4Genomics/Projects/Project_1/ML4G_Project_1_Data/H3K4me1-bigwig/X2.bw
Loading this bigWig file: /Users/gonuni/Desktop/College/CBB/3rd_Semester/ML4Genomics/Projects/Project_1/ML4G_Project_1_Data/H3K4me3-bigwig/X2.bw
Loading this bigWig file: /Users/gonuni/Desktop/College/CBB/3rd_Semester/ML4Genomics/Projects/Project_1/ML4G_Project_1_Data/H3K27ac-

## Model (XGBoost)

In [7]:
# spearmann correlation 
def spearman_corr(y_true, y_pred):
    corr, _ = spearmanr(y_true, y_pred)
    return corr

spearman_scorer = make_scorer(spearman_corr, greater_is_better=True)

# Preapre training and validation data and exclude non-numeric columns 
X_train_df = pd.read_csv('./Data/X_train.tsv', sep='\t')
X_val_df = pd.read_csv('./Data/X_validation.tsv', sep='\t')
non_numeric = ['gene_name', 'gex', 'chr', 'strand']

X_train = X_train_df.drop(columns=non_numeric)
y_train = X_train_df['gex']
X_val = X_val_df.drop(columns=non_numeric)
y_val = X_val_df['gex']

# Define paramteres needed for GridSearch with XGBoost model, fit the model and get best 
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=42
)

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring=spearman_scorer,
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
print("Hyperparameters:", grid_search.best_params_)

y_train_pred = best_model.predict(X_train)
spearman_corr_train = spearmanr(y_train, y_train_pred)[0]
print(f"Correlation on training set: {spearman_corr_train}")

# Get spearman for validation set 
y_val_pred = best_model.predict(X_val)
spearman_corr_val = spearmanr(y_val, y_val_pred)[0]
print(f"Correlation on validation set: {spearman_corr_val}")

Fitting 3 folds for each of 48 candidates, totalling 144 fits
Hyperparameters: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1.0}
Correlation on training set: 0.7795704957519383
Correlation on validation set: 0.7888818601100172


## Predicition on Test Data

In [13]:
X3_test = pd.read_csv('./Data/X_3_test.tsv', sep='\t')
X3_features = X3_test.drop(columns=['gene_name', 'chr', 'strand'])
pred = best_model.predict(X3_features)

assert isinstance(pred, np.ndarray), 'Prediction array must be a numpy array'
assert np.issubdtype(pred.dtype, np.number), 'Prediction array must be numeric'
assert pred.shape[0] == len(X3_test), 'Each gene should have a unique predicted expression'

save_dir = '/Users/gonuni/Desktop/College/CBB/3rd_Semester/ML4Genomics/Projects/ML4Genomics/project1/' 
file_name = 'gex_predicted.csv'        
zip_name = "Indilewitsch_Marie-Claire_Cardenal_Gonzalo_Project1.zip" 
save_path = f'{save_dir}/{zip_name}'
compression_options = dict(method="zip", archive_name=file_name)

test_genes  = pd.DataFrame({
    'gene_name': X3_test['gene_name'],
    'gex_predicted': pred.tolist()
})
test_genes.to_csv(save_path, compression=compression_options)
